In [67]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy


In [73]:
df=pd.read_csv('../../Downloads/current_serviceability_vs_delivery_rate.csv')

In [74]:
df['pincode'] = df['pincode'].str.replace(',', '')

# Convert the 'pincode' column to integers
df['pincode'] = df['pincode'].astype(int)

In [65]:
def get_conn(SSH_requiered,key_path):   #for getting a conn as a result
    
    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_requiered == 'Yes':
        SSH_HOST='ec2-3-111-187-137.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432))
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [68]:
conn = get_conn('Yes','../tunnel-ssh.cer')

Tunnel Started
Connection Made


In [80]:
df2 = df.drop_duplicates(subset=['pincode'], keep="first", inplace=False).set_index('pincode')

In [82]:
df2.to_csv('rate.csv',index='False')

In [ ]:
df = read_sql('query',conn)